In [6]:
import numpy as np
import cv2
import sys
import time
def GaussianPyramid(img, leveln):
    GP = [img]
    for i in range(leveln - 1):
        GP.append(cv2.pyrDown(GP[i]))
    return GP


def LaplacianPyramid(img, leveln):
    LP = []
    for i in range(leveln - 1):
        next_img = cv2.pyrDown(img)
        LP.append(img - cv2.pyrUp(next_img, img.shape[1::-1]))
        img = next_img
    LP.append(img)
    return LP


def blend_pyramid(LPA, LPB, MP):
    blended = []
    for i, M in enumerate(MP):
        blended.append(LPA[i] * M + LPB[i] * (1.0 - M))
    return blended


def reconstruct(LS):
    img = LS[-1]
    for lev_img in LS[-2::-1]:
        img = cv2.pyrUp(img, lev_img.shape[1::-1])
        img += lev_img
    return img

'''
def multi_band_blending(img1, img2, mask, leveln=6):
    max_leveln = int(np.floor(np.log2(min(img1.shape[0], img1.shape[1],
                                          img2.shape[0], img2.shape[1]))))
    if leveln is None:
        leveln = max_leveln
    if leveln < 1 or leveln > max_leveln:
        print ("warning: inappropriate number of leveln")
        leveln = max_leveln
    print(leveln)
    # Get Gaussian pyramid and Laplacian pyramid
    e1 = cv2.getTickCount()
    MP = GaussianPyramid(mask, leveln)
    e2 = cv2.getTickCount()
    print((e2 - e1)/cv2.getTickFrequency()*1000)
    s_LPA = time.time()
    LPA = LaplacianPyramid(img1.astype(np.float64), leveln)
    e_LPA = time.time()
    print(" Time for LPA is %.4f" %(e_LPA-s_LPA))
    s_LPB = time.time()
    LPB = LaplacianPyramid(img2.astype(np.float64), leveln)
    e_LPB = time.time()
    print(" Time for LPB is %.4f" %(e_LPB-s_LPB))
    # Blend two Laplacian pyramidspass
    s_B = time.time()
    blended = blend_pyramid(LPA, LPB, MP)
    e_B = time.time()
    print(" Time for blend_pyramid is %.4f" %(e_B-s_B))
    # Reconstruction process
    s_R = time.time()
    result = reconstruct(blended)
    e_R = time.time()
    print(" Time for reconstruct is %.4f" %(e_R-s_R))
    result[result > 255] = 255
    result[result < 0] = 0

    return result
'''
def multi_band_blending(img1, img2, mask, leveln=6):
    max_leveln = int(np.floor(np.log2(min(img1.shape[0], img1.shape[1],
                                          img2.shape[0], img2.shape[1]))))
    if leveln is None:
        leveln = max_leveln
    if leveln < 1 or leveln > max_leveln:
        print ("warning: inappropriate number of leveln")
        leveln = max_leveln

    # Get Gaussian pyramid and Laplacian pyramid
    MP = GaussianPyramid(mask, leveln)
    LPA = LaplacianPyramid(img1.astype(np.float64), leveln)
    LPB = LaplacianPyramid(img2.astype(np.float64), leveln)
    # Blend two Laplacian pyramidspass
    blended = blend_pyramid(LPA, LPB, MP)

    # Reconstruction process
    result = reconstruct(blended)
    result[result > 255] = 255
    result[result < 0] = 0

    return result

def imgLabeling(img1, img2, img3, img4, maskSize, xoffsetL, xoffsetR,
                minloc_old=None):
    if len(img1.shape) == 3:
        errL = np.sum(np.square(img1.astype(np.float64) -
                                img2.astype(np.float64)), axis=2)
        errR = np.sum(np.square(img3.astype(np.float64) -
                                img4.astype(np.float64)), axis=2)
    else:
        errL = np.square(img1.astype(np.float64) - img2.astype(np.float64))
        errR = np.square(img3.astype(np.float64) - img4.astype(np.float64))
    EL = np.zeros(errL.shape, np.float64)
    ER = np.zeros(errR.shape, np.float64)
    EL[0] = errL[0]
    ER[0] = errR[0]
    for i in range(1, maskSize[1]):
        EL[i, 0] = errL[i, 0] + min(EL[i - 1, 0], EL[i - 1, 1])
        ER[i, 0] = errR[i, 0] + min(ER[i - 1, 0], ER[i - 1, 1])
        for j in range(1, EL.shape[1] - 1):
            EL[i, j] = errL[i, j] + \
                min(EL[i - 1, j - 1], EL[i - 1, j], EL[i - 1, j + 1])
            ER[i, j] = errR[i, j] + \
                min(ER[i - 1, j - 1], ER[i - 1, j], ER[i - 1, j + 1])
        EL[i, -1] = errL[i, -1] + min(EL[i - 1, -1], EL[i - 1, -2])
        ER[i, -1] = errR[i, -1] + min(ER[i - 1, -1], ER[i - 1, -2])
    minlocL = np.argmin(EL, axis=1) + xoffsetL
    minlocR = np.argmin(ER, axis=1) + xoffsetR
    if minloc_old is None:
        minloc_old = [minlocL, minlocR, minlocL, minlocR]
    minlocL_fin = np.int32(0.4 * minlocL + 0.3 *
                           minloc_old[0] + 0.3 * minloc_old[2])
    minlocR_fin = np.int32(0.4 * minlocR + 0.3 *
                           minloc_old[1] + 0.3 * minloc_old[3])
    mask = np.ones((maskSize[1], maskSize[0], 3), np.float64)
    for i in range(maskSize[1]):
        mask[i, minlocL_fin[i]:minlocR_fin[i]] = 0
        mask[i, minlocL_fin[i]] = 0.5
        mask[i, minlocR_fin[i]] = 0.5
    #cv2.imshow('mask', mask.astype(np.float32))
    return mask, [minlocL, minlocR, minlocL_fin, minlocR_fin]

def verticalBoundary(M, W_remap, W, H):
    """Return vertical boundary of input image."""
    row = np.zeros((W_remap, 3, 1))
    row[:, 2] = 1
    row[:, 0] = np.arange(
        (W - W_remap) / 2, (W + W_remap) / 2).reshape((W_remap, 1))
    product = np.matmul(M, row).reshape((W_remap, 3))
    normed = np.array(list(zip(product[:, 0] / product[:, 2], product[:, 1] / product[:, 2])))
    # list(normed)
    top = np.max(
        normed[
            np.logical_and(
                normed[:, 0] >= W_remap / 2,
                normed[:, 0] < W - W_remap / 2)
                ][:, 1])

    row[:, 1] = H - 1
    product = np.matmul(M, row).reshape((W_remap, 3))
    normed = np.array(
        list(zip(product[:, 0] / product[:, 2], product[:, 1] / product[:, 2])))
    bottom = np.min(normed[np.logical_and(
        normed[:, 0] >= W_remap / 2, normed[:, 0] < W - W_remap / 2)][:, 1])

    return int(top) if top > 0 else 0, int(bottom) if bottom < H else H

def getMatches_templmatch(img1, img2, templ_shape, max):
    """Return pairs of corresponding points
    using brute force template matching."""
    if not np.array_equal(img1.shape, img2.shape):
        # print "error: inconsistent array dimention", img1.shape, img2.shape
        print(("error: inconsistent array dimention", img1.shape, img2.shape))
        sys.exit()
    if not (np.all(templ_shape <= img1.shape[:2]) and
            np.all(templ_shape <= img2.shape[:2])):
        print ("error: template shape shall fit img1 and img2")
        sys.exit()

    Hs, Ws = img1.shape[:2]
    Ht, Wt = templ_shape
    matches = []
    for yt in range(0, Hs - Ht + 1, 64):
        for xt in range(0, Ws - Wt + 1, 2):
            result = cv2.matchTemplate(
                img1, img2[yt:yt + Ht, xt:xt + Wt], cv2.TM_CCORR_NORMED)
            minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
            if maxVal > 0.9:
                matches.append((maxVal, maxLoc, (xt, yt)))
    matches.sort(key=lambda e: e[0], reverse=True)
    if len(matches) >= max:
        return np.int32([matches[i][1:] for i in range(max)])
    else:
        return np.int32([c[1:] for c in matches])


def getMatches_goodtemplmatch(img1, img2, templ_shape, max):
    """Return pairs of corresponding points
    using combining Shi-Tomasi corner detector and template matching."""
    if not np.array_equal(img1.shape, img2.shape):
        print(("error: inconsistent array dimention", img1.shape, img2.shape))
        sys.exit()
    if not (np.all(templ_shape <= img1.shape[:2]) and
            np.all(templ_shape <= img2.shape[:2])):
        print ("error: template shape shall fit img1 and img2")
        sys.exit()

    feature_params = dict(maxCorners=max, qualityLevel=0.01,
                          minDistance=5, blockSize=5)
    kps1 = cv2.goodFeaturesToTrack(img1, mask=None, **feature_params)
    kps2 = cv2.goodFeaturesToTrack(img2, mask=None, **feature_params)

    Hs, Ws = img1.shape[:2]
    Ht, Wt = templ_shape
    matches = []
    for [[xt, yt]] in kps1:
        if int(yt) + Ht > Hs or int(xt) + Wt > Ws:
            continue
        result = cv2.matchTemplate(
            img2, img1[int(yt):int(yt) + Ht, int(xt):int(xt) + Wt],
            cv2.TM_CCORR_NORMED)
        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
        if maxVal > 0.9:
            matches.append((maxVal, (int(xt), int(yt)), maxLoc))
    for [[xt, yt]] in kps2:
        if int(yt) + Ht > Hs or int(xt) + Wt > Ws:
            continue
        result = cv2.matchTemplate(
            img1, img2[int(yt):int(yt) + Ht, int(xt):int(xt) + Wt],
            cv2.TM_CCORR_NORMED)
        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
        if maxVal > 0.9:
            matches.append((maxVal, maxLoc, (int(xt), int(yt))))
    matches.sort(key=lambda e: e[0], reverse=True)
    if len(matches) >= max:
        return np.int32([matches[i][1:] for i in range(max)])
    else:
        return np.int32([c[1:] for c in matches])
def equirect_proj(x_proj, y_proj, W, H, fov):
    """Return the equirectangular projection on a unit sphere,
    given cartesian coordinates of the de-warped image."""
    theta_alt = x_proj * fov / W
    phi_alt = y_proj * np.pi / H

    x = np.sin(theta_alt) * np.cos(phi_alt)
    y = np.sin(phi_alt)
    z = np.cos(theta_alt) * np.cos(phi_alt)

    return np.arctan2(y, x), np.arctan2(np.sqrt(x**2 + y**2), z)


def buildmap(Ws, Hs, Wd, Hd, fov):
    """Return a mapping from de-warped images to fisheye images."""
    fov = fov * np.pi / 180.0

    # cartesian coordinates of the de-warped rectangular image
    ys, xs = np.indices((Hs, Ws), np.float32)
    y_proj = Hs / 2.0 - ys
    x_proj = xs - Ws / 2.0

    # spherical coordinates
    theta, phi = equirect_proj(x_proj, y_proj, Ws, Hs, fov)

    # polar coordinates (of the fisheye image)
    p = Hd * phi / fov

    # cartesian coordinates of the fisheye image
    y_fish = p * np.sin(theta)
    x_fish = p * np.cos(theta)

    ymap = Hd / 2.0 - y_fish
    xmap = Wd / 2.0 + x_fish
    return xmap, ymap, p
def Hcalc(cap, xmap, ymap):
    """Calculate and return homography for stitching process."""
    Mlist = []
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    for frame_no in np.arange(0, frame_count, int(frame_count / 10)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
        if ret:
            # defish / unwarp
            cam1 = cv2.remap(frame[:, :1280], xmap, ymap, cv2.INTER_LINEAR)
            cam2 = cv2.remap(frame[:, 1280:], xmap, ymap, cv2.INTER_LINEAR)
            cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
            cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)

            # shift the remapped images along x-axis
            shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
            shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam2
            shifted_cams[:H, :int(W_remap / 2)] = cam1[:, int(W_remap / 2):]
            shifted_cams[:H, W - int(W_remap / 2):] = cam1[:, :int(W_remap / 2)]

            # find matches and extract pairs of correspondent matching points
            matchesL = getMatches_goodtemplmatch(
                cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                templ_shape, maxL)
            matchesR = getMatches_goodtemplmatch(
                cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                templ_shape, maxR)
            matchesR = matchesR[:, -1::-1]

            matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
            matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
            zipped_matches = list(zip(matchesL, matchesR))
            matches = np.int32([e for i in zipped_matches for e in i])
            pts1 = matches[:, 0]
            pts2 = matches[:, 1]

            # find homography from pairs of correspondent matchings
            M, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 5.0)
            Mlist.append(M)
    M = np.average(np.array(Mlist), axis=0)
    print (M)
    #cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    return M
def deduplication(inliers):
    pts_1, indexes_1 = np.unique(inliers[:,1], return_index = True, axis=0)
    label,indexes_2 = np.unique(inliers[:,0][indexes_1], return_index = True, axis=0)
    train = pts_1[indexes_2]
    return train, label

In [6]:
import torch
import torch.optim as optim
from torch import nn
import torch.utils.data as Data

cap1=cv2.VideoCapture('C:/Users/Jackie/Downloads/0129_1.MP4')
cap2=cv2.VideoCapture('C:/Users/Jackie/Downloads/0129_2.MP4')
count=cap1.get(cv2.CAP_PROP_FRAME_COUNT)
fps=cap1.get(cv2.CAP_PROP_FPS)
size=(int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH)*2),int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video=cv2.VideoWriter('C:/Users/Jackie/Downloads/129_improved_blending.MP4',cv2.VideoWriter_fourcc('I','4','2','0'),fps,size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



maxL = 200
maxR = 200

if cap1.isOpened() and cap2.isOpened():
    i=1
    print('successful')
    print("The count is %d" %(count))
    #image size paremeters
    
        
    ret1,frame1=cap1.read()
    ret2,frame2=cap2.read()
    H=frame1.shape[0]
    W_remap=int(194/180*H)
    W=frame1.shape[1]+frame2.shape[1]
    templ_shape = (60, 16)
    offsetYL = int(160/1280*H)
    offsetYR = int(160/1280*H)
    #print(W_remap)
    #remap
    xmap, ymap, p = buildmap(Ws=W_remap, Hs=H, Wd=H, Hd=H, fov=194)
    #cam2 = cv2.remap(frame[:, :H], xmap, ymap, cv2.INTER_LINEAR)
    #cam1 = cv2.remap(frame[:, H:], xmap, ymap, cv2.INTER_LINEAR)
    cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
    cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
    
    #shift
    cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
    cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
    shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
    shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
    shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
    shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
    
    #cv2.imwrite('video_shift.jpg', shifted_cams)
    #matching
    
    maxL = 200
    maxR = 200
    b_matching=time.time()
    matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
    matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
    matchesR = matchesR[:, -1::-1]
    matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
    matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
    zipped_matches = list(zip(matchesL, matchesR))
    matches = np.int32([e for i in zipped_matches for e in i])
    pts1 = matches[:, 0]
    pts2 = matches[:, 1]
    
    #homography
    M_current, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
    
    top, bottom = verticalBoundary(M_current, W_remap, W, H)
    warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
    warped1 = shifted_cams[0:H,:]
    warped1 = cv2.resize(warped1[top:bottom], (W, H))
    warped2 = cv2.resize(warped2[top:bottom], (W, H))
    EAof2 = np.zeros((H, W, 3), np.uint8)
    EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
    EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
    
    #blend
    b_blend=time.time()
    W_lbl = 120
    blend_level = 4
    
    mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
    
    warped1[:, int(W_remap / 2):W - int(W_remap /
                2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
    warped2[EAof2 == 0] = warped1[EAof2 == 0]
    '''
    blended = multi_band_blending(
                    warped1, warped2, mask, blend_level)
    '''
    blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
    blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
    blended = np.ones((H,W,3))
    blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
    blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
    blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
    blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
    blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
    e_blend=time.time()
    #print("Time for blending is %.4f" %(e_blend-b_blend))
    video.write(blended.astype(np.uint8))
    #cv2.imwrite('video_blending.jpg', blended.astype(np.uint8))
    #FCN
    epochs=40
    index=np.where(status.ravel()==1)
    inliers=matches[index[0]]
    train,label = deduplication(inliers)
    zipped_train = list(zip(train, label))
    dataset = np.int32([e for e in zipped_train])
    np.random.shuffle(dataset)
    train = dataset[:, 0]
    label = dataset[:, 1]
    b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
    test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
    label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1)
    M = torch.tensor(M_current,requires_grad=True)
    #M_best = torch.tensor(M_best,requires_grad=True)
    model = nn.Linear(3,3,bias=False)
    model.weight =nn.Parameter(M)
    criterion = nn.MSELoss()
    test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
    label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
    
    test_output = model(test)
    test_output = test_output/test_output[:,2].reshape(-1,1)
    best_loss = criterion(test_output,label_test)/label_test.shape[0]
    
    print("The loss for frame1 is %.4f" %(best_loss))
    #print("The loss of M_best for frame1 is %.4f" %(loss_test_b))
    shape_list = []
    shape_list.append(inliers.shape[0])
    #M_best = M_best.to(device)
    while i<count:
        i+=1
        ret1,frame1=cap1.read()
        ret2,frame2=cap2.read()
        
        if ret1 and ret2:
            
            cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
            cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
            
            #shift
                
            shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
            shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
            shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
            shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
            if len(shape_list)%4==0:
                inliers = np.delete(inliers, range(0,shape_list[0]),axis = 0)
                del(shape_list[0])
             
            if i%60==0:
                cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
                cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
                b_matching=time.time()
                matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
                matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
                matchesR = matchesR[:, -1::-1]
                matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
                matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
                zipped_matches = list(zip(matchesL, matchesR))
                matches_append = np.int32([e for i in zipped_matches for e in i])
                pts1 = matches_append[:, 0]
                pts2 = matches_append[:, 1]
                
                
                e_matching=time.time()
                print("Time for matching is %.4f, round is %d" %((e_matching-b_matching),i) )
                M_, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
                index=np.where(status.ravel()==1)
      
                inliers = np.concatenate((inliers,matches_append[index[0]]),axis=0)
                shape_list.append(matches_append[index[0]].shape[0])
                
        
                train,label = deduplication(inliers)
                zipped_train = list(zip(train, label))
                dataset = np.int32([e for e in zipped_train])
                np.random.shuffle(dataset)
                train = dataset[:, 0]
                label = dataset[:, 1]
                
                b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
                test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                b=np.ones(int(train.shape[0]*0.8))
                train = np.insert(train[:int(train.shape[0]*0.8)], 2, values=b, axis=1) 
                label = np.insert(label[:int(label.shape[0]*0.8)], 2, values=b, axis=1) 
                
                M = torch.tensor(M_current,requires_grad=True)
                model = nn.Linear(3,3,bias=False)
                model.weight =nn.Parameter(M)
                model.to(device)
                criterion = nn.MSELoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
                #optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
                #M = torch.tensor(M,requires_grad=True)
                inputs = torch.tensor(train,requires_grad=False,dtype=torch.float64)
                inputs = inputs.to(device)
                label_t = torch.tensor(label,requires_grad=False,dtype=torch.float64)
                label_t = label_t.to(device)
                test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
                test = test.to(device)
                label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
                label_test = label_test.to(device)
                torch_dataset = Data.TensorDataset(inputs, label_t)

                loader = Data.DataLoader(
                                dataset=torch_dataset,      
                                batch_size=64,      
                                shuffle=True,               
                                num_workers=0,)
                total_step = len(loader)
                t_s = time.time()
                for epoch in range(epochs):
                    for step, (batch_x, l) in enumerate(loader):
                        #batch_x, l = batch_x.to(device),l.to(device)
                        outputs = model(batch_x)
                        outputs = outputs/outputs[:,2].reshape(-1,1)
                        loss = criterion(outputs,l)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, step+1, total_step, loss.item()/len(batch_x)))
                t_e = time.time()
                print(" Time for training is %.4f ms" %((t_e-t_s)*1000))
                test_output = model(test)
                test_output = test_output/test_output[:,2].reshape(-1,1)
                loss_test = criterion(test_output,label_test)/label_test.shape[0]
                
                M_c = torch.tensor(M_current,requires_grad=True)
                model_c = nn.Linear(3,3,bias=False)
                model_c.weight =nn.Parameter(M_c)
                model_c.to(device)
                current_test = model_c(test)
                current_test = current_test/current_test[:,2].reshape(-1,1)
                current_loss = criterion(current_test,label_test)/label_test.shape[0]
                
                print("The loss of M_trained is %.4f, round %d" %(loss_test,i))
                print("The loss of M_current is %.4f, round %d" %(current_loss,i))
                print(model.weight)
                
                if loss_test<current_loss:
                    model = model.cpu()
                    M_current = model.weight.detach().numpy()
                    M_current = M_current/M_current[2][2]
                    best_loss = loss_test
                    print("The best loss is %.4f" %(best_loss))
            
            top, bottom = verticalBoundary(M_current, W_remap, W, H)
            warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
            warped1 = shifted_cams[0:H,:]
            warped1 = cv2.resize(warped1[top:bottom], (W, H))
            warped2 = cv2.resize(warped2[top:bottom], (W, H))
            EAof2 = np.zeros((H, W, 3), np.uint8)
            EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
            EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
            b_blend=time.time()
            
            if i%120==0:
                mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
            
            
            
            warped1[:, int(W_remap / 2):W - int(W_remap /
                        2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
            warped2[EAof2 == 0] = warped1[EAof2 == 0]
            '''
            blended = multi_band_blending(
                                warped1, warped2, mask, blend_level)
            '''
            #blended = warped1 * mask + warped2 * (1 - mask)
            blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
            blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
            blended = np.ones((H,W,3))
            blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
            blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
            blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
            blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
            blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
            video.write(blended.astype(np.uint8))
            e_blend=time.time()
            print(" Time for blending is %.4f, round is %d" %((e_blend-b_blend),i) )
      
        else:   
            break
    
else:
    print('fail')
cap1.release()
cap2.release()
video.release()

successful
The count is 985
The loss for frame1 is 0.0463
 Time for blending is 0.1985, round is 2
 Time for blending is 0.2043, round is 3
 Time for blending is 0.1820, round is 4
 Time for blending is 0.1827, round is 5
 Time for blending is 0.1805, round is 6
 Time for blending is 0.1823, round is 7
 Time for blending is 0.2073, round is 8
 Time for blending is 0.2007, round is 9
 Time for blending is 0.2314, round is 10
 Time for blending is 0.2347, round is 11
 Time for blending is 0.2268, round is 12
 Time for blending is 0.2177, round is 13
 Time for blending is 0.2349, round is 14
 Time for blending is 0.2073, round is 15
 Time for blending is 0.2229, round is 16
 Time for blending is 0.2055, round is 17
 Time for blending is 0.2408, round is 18
 Time for blending is 0.2587, round is 19
 Time for blending is 0.2827, round is 20
 Time for blending is 0.1785, round is 21
 Time for blending is 0.2197, round is 22
 Time for blending is 0.2241, round is 23
 Time for blending is 0.17

 Time for blending is 0.2193, round is 60
 Time for blending is 0.2002, round is 61
 Time for blending is 0.2227, round is 62
 Time for blending is 0.2031, round is 63
 Time for blending is 0.2079, round is 64
 Time for blending is 0.2277, round is 65
 Time for blending is 0.1919, round is 66
 Time for blending is 0.1749, round is 67
 Time for blending is 0.1875, round is 68
 Time for blending is 0.1897, round is 69
 Time for blending is 0.1738, round is 70
 Time for blending is 0.1896, round is 71
 Time for blending is 0.1879, round is 72
 Time for blending is 0.1899, round is 73
 Time for blending is 0.1861, round is 74
 Time for blending is 0.1733, round is 75
 Time for blending is 0.1761, round is 76
 Time for blending is 0.1869, round is 77
 Time for blending is 0.1844, round is 78
 Time for blending is 0.1727, round is 79
 Time for blending is 0.1885, round is 80
 Time for blending is 0.1898, round is 81
 Time for blending is 0.1748, round is 82
 Time for blending is 0.1875, roun

 Time for blending is 0.5360, round is 120
 Time for blending is 0.1761, round is 121
 Time for blending is 0.1873, round is 122
 Time for blending is 0.1896, round is 123
 Time for blending is 0.1740, round is 124
 Time for blending is 0.1739, round is 125
 Time for blending is 0.1745, round is 126
 Time for blending is 0.1875, round is 127
 Time for blending is 0.1919, round is 128
 Time for blending is 0.1908, round is 129
 Time for blending is 0.1879, round is 130
 Time for blending is 0.1755, round is 131
 Time for blending is 0.1868, round is 132
 Time for blending is 0.1896, round is 133
 Time for blending is 0.1953, round is 134
 Time for blending is 0.1879, round is 135
 Time for blending is 0.2007, round is 136
 Time for blending is 0.1920, round is 137
 Time for blending is 0.1893, round is 138
 Time for blending is 0.1982, round is 139
 Time for blending is 0.1802, round is 140
 Time for blending is 0.1841, round is 141
 Time for blending is 0.1978, round is 142
 Time for b

Epoch [32/40], Step [5/6], Loss: 0.0328
Epoch [32/40], Step [6/6], Loss: 0.0614
Epoch [33/40], Step [1/6], Loss: 0.0280
Epoch [33/40], Step [2/6], Loss: 0.0400
Epoch [33/40], Step [3/6], Loss: 0.0288
Epoch [33/40], Step [4/6], Loss: 0.0336
Epoch [33/40], Step [5/6], Loss: 0.0383
Epoch [33/40], Step [6/6], Loss: 0.0624
Epoch [34/40], Step [1/6], Loss: 0.0294
Epoch [34/40], Step [2/6], Loss: 0.0308
Epoch [34/40], Step [3/6], Loss: 0.0380
Epoch [34/40], Step [4/6], Loss: 0.0324
Epoch [34/40], Step [5/6], Loss: 0.0388
Epoch [34/40], Step [6/6], Loss: 0.0585
Epoch [35/40], Step [1/6], Loss: 0.0342
Epoch [35/40], Step [2/6], Loss: 0.0370
Epoch [35/40], Step [3/6], Loss: 0.0360
Epoch [35/40], Step [4/6], Loss: 0.0271
Epoch [35/40], Step [5/6], Loss: 0.0381
Epoch [35/40], Step [6/6], Loss: 0.0488
Epoch [36/40], Step [1/6], Loss: 0.0217
Epoch [36/40], Step [2/6], Loss: 0.0236
Epoch [36/40], Step [3/6], Loss: 0.0326
Epoch [36/40], Step [4/6], Loss: 0.0375
Epoch [36/40], Step [5/6], Loss: 0.0479


Epoch [20/40], Step [4/6], Loss: 0.0550
Epoch [20/40], Step [5/6], Loss: 0.0521
Epoch [20/40], Step [6/6], Loss: 0.1656
Epoch [21/40], Step [1/6], Loss: 0.0484
Epoch [21/40], Step [2/6], Loss: 0.0635
Epoch [21/40], Step [3/6], Loss: 0.0555
Epoch [21/40], Step [4/6], Loss: 0.0215
Epoch [21/40], Step [5/6], Loss: 0.0700
Epoch [21/40], Step [6/6], Loss: 0.1377
Epoch [22/40], Step [1/6], Loss: 0.0556
Epoch [22/40], Step [2/6], Loss: 0.0520
Epoch [22/40], Step [3/6], Loss: 0.0677
Epoch [22/40], Step [4/6], Loss: 0.0303
Epoch [22/40], Step [5/6], Loss: 0.0691
Epoch [22/40], Step [6/6], Loss: 0.1324
Epoch [23/40], Step [1/6], Loss: 0.0617
Epoch [23/40], Step [2/6], Loss: 0.0723
Epoch [23/40], Step [3/6], Loss: 0.0535
Epoch [23/40], Step [4/6], Loss: 0.0431
Epoch [23/40], Step [5/6], Loss: 0.0622
Epoch [23/40], Step [6/6], Loss: 0.1516
Epoch [24/40], Step [1/6], Loss: 0.0343
Epoch [24/40], Step [2/6], Loss: 0.0519
Epoch [24/40], Step [3/6], Loss: 0.0607
Epoch [24/40], Step [4/6], Loss: 0.0568


...

Epoch [38/40], Step [1/5], Loss: 0.2809
Epoch [38/40], Step [2/5], Loss: 0.3299
Epoch [38/40], Step [3/5], Loss: 0.3496
Epoch [38/40], Step [4/5], Loss: 0.2278
Epoch [38/40], Step [5/5], Loss: 0.2929
Epoch [39/40], Step [1/5], Loss: 0.3356
Epoch [39/40], Step [2/5], Loss: 0.2978
Epoch [39/40], Step [3/5], Loss: 0.2675
Epoch [39/40], Step [4/5], Loss: 0.2618
Epoch [39/40], Step [5/5], Loss: 0.3159
Epoch [40/40], Step [1/5], Loss: 0.2945
Epoch [40/40], Step [2/5], Loss: 0.2207
Epoch [40/40], Step [3/5], Loss: 0.3458
Epoch [40/40], Step [4/5], Loss: 0.2541
Epoch [40/40], Step [5/5], Loss: 0.3701
 Time for training is 594.9004 ms
The loss of M_trained is 0.2215, round 720
The loss of M_current is 0.2288, round 720
Parameter containing:
tensor([[ 1.0748e+00,  5.0756e-02, -1.4992e+02],
        [-6.5659e-03,  1.0533e+00, -1.3712e+01],
        [-8.1202e-08,  2.2556e-05,  9.9998e-01]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)
The best loss is 0.2215
 Time for blending is

Epoch [40/40], Step [4/5], Loss: 0.2862
Epoch [40/40], Step [5/5], Loss: 0.3184
 Time for training is 561.0719 ms
The loss of M_trained is 0.2267, round 780
The loss of M_current is 0.2330, round 780
Parameter containing:
tensor([[ 1.0749e+00,  5.0905e-02, -1.4992e+02],
        [-6.5238e-03,  1.0531e+00, -1.3712e+01],
        [-3.6915e-08,  2.2518e-05,  9.9998e-01]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)
The best loss is 0.2267
 Time for blending is 0.1869, round is 780
 Time for blending is 0.1878, round is 781
 Time for blending is 0.1731, round is 782
 Time for blending is 0.1880, round is 783
 Time for blending is 0.1898, round is 784
 Time for blending is 0.1910, round is 785
 Time for blending is 0.1895, round is 786
 Time for blending is 0.2055, round is 787
 Time for blending is 0.1909, round is 788
 Time for blending is 0.1898, round is 789
 Time for blending is 0.1880, round is 790
 Time for blending is 0.2068, round is 791
 Time for blending is 0.1

Epoch [40/40], Step [5/5], Loss: 0.2053
 Time for training is 588.9094 ms
The loss of M_trained is 0.2301, round 840
The loss of M_current is 0.2339, round 840
Parameter containing:
tensor([[ 1.0750e+00,  5.1047e-02, -1.4992e+02],
        [-6.4410e-03,  1.0530e+00, -1.3712e+01],
        [-2.0960e-07,  2.2712e-05,  9.9998e-01]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)
The best loss is 0.2301
 Time for blending is 0.5413, round is 840
 Time for blending is 0.1774, round is 841
 Time for blending is 0.1859, round is 842
 Time for blending is 0.1981, round is 843
 Time for blending is 0.1907, round is 844
 Time for blending is 0.1904, round is 845
 Time for blending is 0.1885, round is 846
 Time for blending is 0.1731, round is 847
 Time for blending is 0.2064, round is 848
 Time for blending is 0.2042, round is 849
 Time for blending is 0.1766, round is 850
 Time for blending is 0.1875, round is 851
 Time for blending is 0.1907, round is 852
 Time for blending is 

Epoch [38/40], Step [4/6], Loss: 0.2540
Epoch [38/40], Step [5/6], Loss: 0.2360
Epoch [38/40], Step [6/6], Loss: 0.5018
Epoch [39/40], Step [1/6], Loss: 0.2457
Epoch [39/40], Step [2/6], Loss: 0.2218
Epoch [39/40], Step [3/6], Loss: 0.2248
Epoch [39/40], Step [4/6], Loss: 0.2748
Epoch [39/40], Step [5/6], Loss: 0.2343
Epoch [39/40], Step [6/6], Loss: 0.9276
Epoch [40/40], Step [1/6], Loss: 0.1791
Epoch [40/40], Step [2/6], Loss: 0.2247
Epoch [40/40], Step [3/6], Loss: 0.2420
Epoch [40/40], Step [4/6], Loss: 0.2792
Epoch [40/40], Step [5/6], Loss: 0.2778
Epoch [40/40], Step [6/6], Loss: 0.8177
 Time for training is 580.3423 ms
The loss of M_trained is 0.1943, round 900
The loss of M_current is 0.2041, round 900
Parameter containing:
tensor([[ 1.0751e+00,  5.1208e-02, -1.4993e+02],
        [-6.3288e-03,  1.0528e+00, -1.3713e+01],
        [-1.5359e-07,  2.2972e-05,  9.9998e-01]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)
The best loss is 0.1943
 Time for blending is

Epoch [38/40], Step [6/6], Loss: 0.3326
Epoch [39/40], Step [1/6], Loss: 0.2594
Epoch [39/40], Step [2/6], Loss: 0.2527
Epoch [39/40], Step [3/6], Loss: 0.1952
Epoch [39/40], Step [4/6], Loss: 0.2406
Epoch [39/40], Step [5/6], Loss: 0.1852
Epoch [39/40], Step [6/6], Loss: 0.5844
Epoch [40/40], Step [1/6], Loss: 0.2155
Epoch [40/40], Step [2/6], Loss: 0.2740
Epoch [40/40], Step [3/6], Loss: 0.2187
Epoch [40/40], Step [4/6], Loss: 0.2015
Epoch [40/40], Step [5/6], Loss: 0.1984
Epoch [40/40], Step [6/6], Loss: 0.6487
 Time for training is 590.7028 ms
The loss of M_trained is 0.1788, round 960
The loss of M_current is 0.1848, round 960
Parameter containing:
tensor([[ 1.0752e+00,  5.1383e-02, -1.4993e+02],
        [-6.2432e-03,  1.0526e+00, -1.3713e+01],
        [-5.9347e-08,  2.2901e-05,  9.9998e-01]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)
The best loss is 0.1788
 Time for blending is 0.5646, round is 960
 Time for blending is 0.1746, round is 961
 Time for blend

In [8]:
import torch
import torch.optim as optim
from torch import nn
import torch.utils.data as Data

cap1=cv2.VideoCapture('C:/Users/Jackie/Downloads/0121_1.MP4')
cap2=cv2.VideoCapture('C:/Users/Jackie/Downloads/0121_2.MP4')
count=cap1.get(cv2.CAP_PROP_FRAME_COUNT)
fps=cap1.get(cv2.CAP_PROP_FPS)
size=(int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH)*2),int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video=cv2.VideoWriter('C:/Users/Jackie/Downloads/121_improved_blending_maskrate240.MP4',cv2.VideoWriter_fourcc('I','4','2','0'),fps,size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



maxL = 200
maxR = 200

if cap1.isOpened() and cap2.isOpened():
    i=1
    print('successful')
    print("The count is %d" %(count))
    #image size paremeters
    
        
    ret1,frame1=cap1.read()
    ret2,frame2=cap2.read()
    H=frame1.shape[0]
    W_remap=int(194/180*H)
    W=frame1.shape[1]+frame2.shape[1]
    templ_shape = (60, 16)
    offsetYL = int(160/1280*H)
    offsetYR = int(160/1280*H)
    #print(W_remap)
    #remap
    xmap, ymap, p = buildmap(Ws=W_remap, Hs=H, Wd=H, Hd=H, fov=194)
    #cam2 = cv2.remap(frame[:, :H], xmap, ymap, cv2.INTER_LINEAR)
    #cam1 = cv2.remap(frame[:, H:], xmap, ymap, cv2.INTER_LINEAR)
    cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
    cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
    
    #shift
    cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
    cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
    shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
    shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
    shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
    shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
    
    #cv2.imwrite('video_shift.jpg', shifted_cams)
    #matching
    
    maxL = 200
    maxR = 200
    b_matching=time.time()
    matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
    matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
    matchesR = matchesR[:, -1::-1]
    matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
    matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
    zipped_matches = list(zip(matchesL, matchesR))
    matches = np.int32([e for i in zipped_matches for e in i])
    pts1 = matches[:, 0]
    pts2 = matches[:, 1]
    
    #homography
    M_current, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
    
    top, bottom = verticalBoundary(M_current, W_remap, W, H)
    warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
    warped1 = shifted_cams[0:H,:]
    warped1 = cv2.resize(warped1[top:bottom], (W, H))
    warped2 = cv2.resize(warped2[top:bottom], (W, H))
    EAof2 = np.zeros((H, W, 3), np.uint8)
    EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
    EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
    
    #blend
    b_blend=time.time()
    W_lbl = 120
    blend_level = 4
    
    mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
    
    warped1[:, int(W_remap / 2):W - int(W_remap /
                2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
    warped2[EAof2 == 0] = warped1[EAof2 == 0]
    '''
    blended = multi_band_blending(
                    warped1, warped2, mask, blend_level)
    '''
    blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
    blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
    blended = np.ones((H,W,3))
    blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
    blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
    blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
    blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
    blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
    e_blend=time.time()
    #print("Time for blending is %.4f" %(e_blend-b_blend))
    video.write(blended.astype(np.uint8))
    #cv2.imwrite('video_blending.jpg', blended.astype(np.uint8))
    #FCN
    epochs=40
    index=np.where(status.ravel()==1)
    inliers=matches[index[0]]
    train,label = deduplication(inliers)
    zipped_train = list(zip(train, label))
    dataset = np.int32([e for e in zipped_train])
    np.random.shuffle(dataset)
    train = dataset[:, 0]
    label = dataset[:, 1]
    b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
    test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
    label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1)
    M = torch.tensor(M_current,requires_grad=True)
    #M_best = torch.tensor(M_best,requires_grad=True)
    model = nn.Linear(3,3,bias=False)
    model.weight =nn.Parameter(M)
    criterion = nn.MSELoss()
    test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
    label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
    
    test_output = model(test)
    test_output = test_output/test_output[:,2].reshape(-1,1)
    best_loss = criterion(test_output,label_test)/label_test.shape[0]
    
    print("The loss for frame1 is %.4f" %(best_loss))
    #print("The loss of M_best for frame1 is %.4f" %(loss_test_b))
    shape_list = []
    shape_list.append(inliers.shape[0])
    #M_best = M_best.to(device)
    while i<count:
        i+=1
        ret1,frame1=cap1.read()
        ret2,frame2=cap2.read()
        
        if ret1 and ret2:
            
            cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
            cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
            
            #shift
                
            shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
            shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
            shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
            shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
            if len(shape_list)%4==0:
                inliers = np.delete(inliers, range(0,shape_list[0]),axis = 0)
                del(shape_list[0])
             
            if i%60==0:
                cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
                cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
                b_matching=time.time()
                matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
                matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
                matchesR = matchesR[:, -1::-1]
                matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
                matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
                zipped_matches = list(zip(matchesL, matchesR))
                matches_append = np.int32([e for i in zipped_matches for e in i])
                pts1 = matches_append[:, 0]
                pts2 = matches_append[:, 1]
                
                
                e_matching=time.time()
                print("Time for matching is %.4f, round is %d" %((e_matching-b_matching),i) )
                M_, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
                index=np.where(status.ravel()==1)
      
                inliers = np.concatenate((inliers,matches_append[index[0]]),axis=0)
                shape_list.append(matches_append[index[0]].shape[0])
                
        
                train,label = deduplication(inliers)
                zipped_train = list(zip(train, label))
                dataset = np.int32([e for e in zipped_train])
                np.random.shuffle(dataset)
                train = dataset[:, 0]
                label = dataset[:, 1]
                
                b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
                test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                b=np.ones(int(train.shape[0]*0.8))
                train = np.insert(train[:int(train.shape[0]*0.8)], 2, values=b, axis=1) 
                label = np.insert(label[:int(label.shape[0]*0.8)], 2, values=b, axis=1) 
                
                M = torch.tensor(M_current,requires_grad=True)
                model = nn.Linear(3,3,bias=False)
                model.weight =nn.Parameter(M)
                model.to(device)
                criterion = nn.MSELoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
                #optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
                #M = torch.tensor(M,requires_grad=True)
                inputs = torch.tensor(train,requires_grad=False,dtype=torch.float64)
                inputs = inputs.to(device)
                label_t = torch.tensor(label,requires_grad=False,dtype=torch.float64)
                label_t = label_t.to(device)
                test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
                test = test.to(device)
                label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
                label_test = label_test.to(device)
                torch_dataset = Data.TensorDataset(inputs, label_t)

                loader = Data.DataLoader(
                                dataset=torch_dataset,      
                                batch_size=64,      
                                shuffle=True,               
                                num_workers=0,)
                total_step = len(loader)
                t_s = time.time()
                for epoch in range(epochs):
                    for step, (batch_x, l) in enumerate(loader):
                        #batch_x, l = batch_x.to(device),l.to(device)
                        outputs = model(batch_x)
                        outputs = outputs/outputs[:,2].reshape(-1,1)
                        loss = criterion(outputs,l)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, step+1, total_step, loss.item()/len(batch_x)))
                t_e = time.time()
                print(" Time for training is %.4f ms" %((t_e-t_s)*1000))
                test_output = model(test)
                test_output = test_output/test_output[:,2].reshape(-1,1)
                loss_test = criterion(test_output,label_test)/label_test.shape[0]
                
                M_c = torch.tensor(M_current,requires_grad=True)
                model_c = nn.Linear(3,3,bias=False)
                model_c.weight =nn.Parameter(M_c)
                model_c.to(device)
                current_test = model_c(test)
                current_test = current_test/current_test[:,2].reshape(-1,1)
                current_loss = criterion(current_test,label_test)/label_test.shape[0]
                
                print("The loss of M_trained is %.4f, round %d" %(loss_test,i))
                print("The loss of M_current is %.4f, round %d" %(current_loss,i))
                print(model.weight)
                
                if loss_test<current_loss:
                    model = model.cpu()
                    M_current = model.weight.detach().numpy()
                    M_current = M_current/M_current[2][2]
                    best_loss = loss_test
                    print("The best loss is %.4f" %(best_loss))
            
            top, bottom = verticalBoundary(M_current, W_remap, W, H)
            warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
            warped1 = shifted_cams[0:H,:]
            warped1 = cv2.resize(warped1[top:bottom], (W, H))
            warped2 = cv2.resize(warped2[top:bottom], (W, H))
            EAof2 = np.zeros((H, W, 3), np.uint8)
            EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
            EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
            b_blend=time.time()
            
            if i%240==0:
                mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
            
            
            
            warped1[:, int(W_remap / 2):W - int(W_remap /
                        2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
            warped2[EAof2 == 0] = warped1[EAof2 == 0]
            '''
            blended = multi_band_blending(
                                warped1, warped2, mask, blend_level)
            '''
            #blended = warped1 * mask + warped2 * (1 - mask)
            blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
            blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
            blended = np.ones((H,W,3))
            blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
            blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
            blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
            blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
            blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
            video.write(blended.astype(np.uint8))
            e_blend=time.time()
            print(" Time for blending is %.4f, round is %d" %((e_blend-b_blend),i) )
      
        else:   
            break
    
else:
    print('fail')
cap1.release()
cap2.release()
video.release()

successful
The count is 943
The loss for frame1 is 0.0699
 Time for blending is 0.3210, round is 2
 Time for blending is 0.3109, round is 3
 Time for blending is 0.3098, round is 4
 Time for blending is 0.3116, round is 5
 Time for blending is 0.3410, round is 6
 Time for blending is 0.3126, round is 7
 Time for blending is 0.3108, round is 8
 Time for blending is 0.2952, round is 9
 Time for blending is 0.2966, round is 10
 Time for blending is 0.2929, round is 11
 Time for blending is 0.2952, round is 12
 Time for blending is 0.3109, round is 13
 Time for blending is 0.3110, round is 14
 Time for blending is 0.3176, round is 15
 Time for blending is 0.3158, round is 16
 Time for blending is 0.3066, round is 17
 Time for blending is 0.3253, round is 18
 Time for blending is 0.3426, round is 19
 Time for blending is 0.3159, round is 20
 Time for blending is 0.3096, round is 21
 Time for blending is 0.3113, round is 22
 Time for blending is 0.3117, round is 23
 Time for blending is 0.31

 Time for blending is 0.3092, round is 73
 Time for blending is 0.3115, round is 74
 Time for blending is 0.3112, round is 75
 Time for blending is 0.3108, round is 76
 Time for blending is 0.3267, round is 77
 Time for blending is 0.3117, round is 78
 Time for blending is 0.3099, round is 79
 Time for blending is 0.3112, round is 80
 Time for blending is 0.3126, round is 81
 Time for blending is 0.3261, round is 82
 Time for blending is 0.3275, round is 83
 Time for blending is 0.3118, round is 84
 Time for blending is 0.3099, round is 85
 Time for blending is 0.3275, round is 86
 Time for blending is 0.3276, round is 87
 Time for blending is 0.3255, round is 88
 Time for blending is 0.3286, round is 89
 Time for blending is 0.3275, round is 90
 Time for blending is 0.2961, round is 91
 Time for blending is 0.3101, round is 92
 Time for blending is 0.3115, round is 93
 Time for blending is 0.2942, round is 94
 Time for blending is 0.3108, round is 95
 Time for blending is 0.3095, roun

 Time for blending is 0.2988, round is 120
 Time for blending is 0.3105, round is 121
 Time for blending is 0.3117, round is 122
 Time for blending is 0.3100, round is 123
 Time for blending is 0.3102, round is 124
 Time for blending is 0.2978, round is 125
 Time for blending is 0.3076, round is 126
 Time for blending is 0.3134, round is 127
 Time for blending is 0.3103, round is 128
 Time for blending is 0.3107, round is 129
 Time for blending is 0.3107, round is 130
 Time for blending is 0.3120, round is 131
 Time for blending is 0.3275, round is 132
 Time for blending is 0.3096, round is 133
 Time for blending is 0.3104, round is 134
 Time for blending is 0.2949, round is 135
 Time for blending is 0.3108, round is 136
 Time for blending is 0.2958, round is 137
 Time for blending is 0.3114, round is 138
 Time for blending is 0.3101, round is 139
 Time for blending is 0.3105, round is 140
 Time for blending is 0.2927, round is 141
 Time for blending is 0.3102, round is 142
 Time for b

 Time for blending is 0.3053, round is 180
 Time for blending is 0.3150, round is 181
 Time for blending is 0.3098, round is 182
 Time for blending is 0.3110, round is 183
 Time for blending is 0.3282, round is 184
 Time for blending is 0.3108, round is 185
 Time for blending is 0.3131, round is 186
 Time for blending is 0.3255, round is 187
 Time for blending is 0.3097, round is 188
 Time for blending is 0.3151, round is 189
 Time for blending is 0.3240, round is 190
 Time for blending is 0.2962, round is 191
 Time for blending is 0.3113, round is 192
 Time for blending is 0.3118, round is 193
 Time for blending is 0.3244, round is 194
 Time for blending is 0.3126, round is 195
 Time for blending is 0.3148, round is 196
 Time for blending is 0.3256, round is 197
 Time for blending is 0.3125, round is 198
 Time for blending is 0.3115, round is 199
 Time for blending is 0.3087, round is 200
 Time for blending is 0.3123, round is 201
 Time for blending is 0.3103, round is 202
 Time for b

 Time for blending is 0.8145, round is 240
 Time for blending is 0.3060, round is 241
 Time for blending is 0.3717, round is 242
 Time for blending is 0.3176, round is 243
 Time for blending is 0.3088, round is 244
 Time for blending is 0.2961, round is 245
 Time for blending is 0.3102, round is 246
 Time for blending is 0.2959, round is 247
 Time for blending is 0.3106, round is 248
 Time for blending is 0.2960, round is 249
 Time for blending is 0.3082, round is 250
 Time for blending is 0.3137, round is 251
 Time for blending is 0.2941, round is 252
 Time for blending is 0.2946, round is 253
 Time for blending is 0.3116, round is 254
 Time for blending is 0.3101, round is 255
 Time for blending is 0.3127, round is 256
 Time for blending is 0.3097, round is 257
 Time for blending is 0.3127, round is 258
 Time for blending is 0.3267, round is 259
 Time for blending is 0.3127, round is 260
 Time for blending is 0.3109, round is 261
 Time for blending is 0.3097, round is 262
 Time for b

...

 Time for blending is 0.2970, round is 660
 Time for blending is 0.3012, round is 661
 Time for blending is 0.3171, round is 662
 Time for blending is 0.3168, round is 663
 Time for blending is 0.3167, round is 664
 Time for blending is 0.3092, round is 665
 Time for blending is 0.3166, round is 666
 Time for blending is 0.3167, round is 667
 Time for blending is 0.3317, round is 668
 Time for blending is 0.3014, round is 669
 Time for blending is 0.3012, round is 670
 Time for blending is 0.3008, round is 671
 Time for blending is 0.3009, round is 672
 Time for blending is 0.3169, round is 673
 Time for blending is 0.3165, round is 674
 Time for blending is 0.3168, round is 675
 Time for blending is 0.3013, round is 676
 Time for blending is 0.3159, round is 677
 Time for blending is 0.3165, round is 678
 Time for blending is 0.3012, round is 679
 Time for blending is 0.3015, round is 680
 Time for blending is 0.3168, round is 681
 Time for blending is 0.3177, round is 682
 Time for b

Epoch [31/40], Step [2/5], Loss: 0.1804
Epoch [31/40], Step [3/5], Loss: 0.4509
Epoch [31/40], Step [4/5], Loss: 0.1841
Epoch [31/40], Step [5/5], Loss: 1.1828
Epoch [32/40], Step [1/5], Loss: 0.2936
Epoch [32/40], Step [2/5], Loss: 0.2453
Epoch [32/40], Step [3/5], Loss: 0.2739
Epoch [32/40], Step [4/5], Loss: 0.1491
Epoch [32/40], Step [5/5], Loss: 3.9067
Epoch [33/40], Step [1/5], Loss: 0.3022
Epoch [33/40], Step [2/5], Loss: 0.3813
Epoch [33/40], Step [3/5], Loss: 0.2829
Epoch [33/40], Step [4/5], Loss: 0.2173
Epoch [33/40], Step [5/5], Loss: 1.2236
Epoch [34/40], Step [1/5], Loss: 0.2549
Epoch [34/40], Step [2/5], Loss: 0.2243
Epoch [34/40], Step [3/5], Loss: 0.2430
Epoch [34/40], Step [4/5], Loss: 0.2699
Epoch [34/40], Step [5/5], Loss: 9.7488
Epoch [35/40], Step [1/5], Loss: 0.2654
Epoch [35/40], Step [2/5], Loss: 0.2904
Epoch [35/40], Step [3/5], Loss: 0.1554
Epoch [35/40], Step [4/5], Loss: 0.3094
Epoch [35/40], Step [5/5], Loss: 4.2420
Epoch [36/40], Step [1/5], Loss: 0.1301


Epoch [30/40], Step [3/5], Loss: 0.1292
Epoch [30/40], Step [4/5], Loss: 0.2844
Epoch [30/40], Step [5/5], Loss: 1.7756
Epoch [31/40], Step [1/5], Loss: 0.2071
Epoch [31/40], Step [2/5], Loss: 0.2174
Epoch [31/40], Step [3/5], Loss: 0.2805
Epoch [31/40], Step [4/5], Loss: 0.2122
Epoch [31/40], Step [5/5], Loss: 0.1451
Epoch [32/40], Step [1/5], Loss: 0.1641
Epoch [32/40], Step [2/5], Loss: 0.2160
Epoch [32/40], Step [3/5], Loss: 0.3148
Epoch [32/40], Step [4/5], Loss: 0.1772
Epoch [32/40], Step [5/5], Loss: 7.8707
Epoch [33/40], Step [1/5], Loss: 0.1206
Epoch [33/40], Step [2/5], Loss: 0.2580
Epoch [33/40], Step [3/5], Loss: 0.2240
Epoch [33/40], Step [4/5], Loss: 0.2912
Epoch [33/40], Step [5/5], Loss: 0.7946
Epoch [34/40], Step [1/5], Loss: 0.2050
Epoch [34/40], Step [2/5], Loss: 0.2308
Epoch [34/40], Step [3/5], Loss: 0.1692
Epoch [34/40], Step [4/5], Loss: 0.2792
Epoch [34/40], Step [5/5], Loss: 5.0978
Epoch [35/40], Step [1/5], Loss: 0.2971
Epoch [35/40], Step [2/5], Loss: 0.1680


Epoch [26/40], Step [4/4], Loss: 0.2866
Epoch [27/40], Step [1/4], Loss: 0.3057
Epoch [27/40], Step [2/4], Loss: 0.2562
Epoch [27/40], Step [3/4], Loss: 0.1694
Epoch [27/40], Step [4/4], Loss: 0.2109
Epoch [28/40], Step [1/4], Loss: 0.2796
Epoch [28/40], Step [2/4], Loss: 0.2049
Epoch [28/40], Step [3/4], Loss: 0.1961
Epoch [28/40], Step [4/4], Loss: 0.2651
Epoch [29/40], Step [1/4], Loss: 0.1933
Epoch [29/40], Step [2/4], Loss: 0.2692
Epoch [29/40], Step [3/4], Loss: 0.1761
Epoch [29/40], Step [4/4], Loss: 0.3155
Epoch [30/40], Step [1/4], Loss: 0.1557
Epoch [30/40], Step [2/4], Loss: 0.2141
Epoch [30/40], Step [3/4], Loss: 0.3098
Epoch [30/40], Step [4/4], Loss: 0.2861
Epoch [31/40], Step [1/4], Loss: 0.2081
Epoch [31/40], Step [2/4], Loss: 0.1823
Epoch [31/40], Step [3/4], Loss: 0.3413
Epoch [31/40], Step [4/4], Loss: 0.2306
Epoch [32/40], Step [1/4], Loss: 0.1489
Epoch [32/40], Step [2/4], Loss: 0.2230
Epoch [32/40], Step [3/4], Loss: 0.3185
Epoch [32/40], Step [4/4], Loss: 0.2755


 Time for blending is 0.3127, round is 900
 Time for blending is 0.3134, round is 901
 Time for blending is 0.3171, round is 902
 Time for blending is 0.2849, round is 903
 Time for blending is 0.3011, round is 904
 Time for blending is 0.3171, round is 905
 Time for blending is 0.3172, round is 906
 Time for blending is 0.3166, round is 907
 Time for blending is 0.3338, round is 908
 Time for blending is 0.3050, round is 909
 Time for blending is 0.2904, round is 910
 Time for blending is 0.3021, round is 911
 Time for blending is 0.3007, round is 912
 Time for blending is 0.3016, round is 913
 Time for blending is 0.3014, round is 914
 Time for blending is 0.3015, round is 915
 Time for blending is 0.3058, round is 916
 Time for blending is 0.3044, round is 917
 Time for blending is 0.3010, round is 918
 Time for blending is 0.3219, round is 919
 Time for blending is 0.3168, round is 920
 Time for blending is 0.3125, round is 921
 Time for blending is 0.3180, round is 922
 Time for b

In [9]:
import torch
import torch.optim as optim
from torch import nn
import torch.utils.data as Data

cap1=cv2.VideoCapture('C:/Users/Jackie/Downloads/0122_1.MP4')
cap2=cv2.VideoCapture('C:/Users/Jackie/Downloads/0122_2.MP4')
count=cap1.get(cv2.CAP_PROP_FRAME_COUNT)
fps=cap1.get(cv2.CAP_PROP_FPS)
size=(int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH)*2),int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video=cv2.VideoWriter('C:/Users/Jackie/Downloads/122_improved_blending_maskrate240.MP4',cv2.VideoWriter_fourcc('I','4','2','0'),fps,size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



maxL = 200
maxR = 200

if cap1.isOpened() and cap2.isOpened():
    i=1
    print('successful')
    print("The count is %d" %(count))
    #image size paremeters
    
        
    ret1,frame1=cap1.read()
    ret2,frame2=cap2.read()
    H=frame1.shape[0]
    W_remap=int(194/180*H)
    W=frame1.shape[1]+frame2.shape[1]
    templ_shape = (60, 16)
    offsetYL = int(160/1280*H)
    offsetYR = int(160/1280*H)
    #print(W_remap)
    #remap
    xmap, ymap, p = buildmap(Ws=W_remap, Hs=H, Wd=H, Hd=H, fov=194)
    #cam2 = cv2.remap(frame[:, :H], xmap, ymap, cv2.INTER_LINEAR)
    #cam1 = cv2.remap(frame[:, H:], xmap, ymap, cv2.INTER_LINEAR)
    cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
    cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
    
    #shift
    cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
    cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
    shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
    shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
    shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
    shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
    
    #cv2.imwrite('video_shift.jpg', shifted_cams)
    #matching
    
    maxL = 200
    maxR = 200
    b_matching=time.time()
    matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
    matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
    matchesR = matchesR[:, -1::-1]
    matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
    matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
    zipped_matches = list(zip(matchesL, matchesR))
    matches = np.int32([e for i in zipped_matches for e in i])
    pts1 = matches[:, 0]
    pts2 = matches[:, 1]
    
    #homography
    M_current, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
    
    top, bottom = verticalBoundary(M_current, W_remap, W, H)
    warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
    warped1 = shifted_cams[0:H,:]
    warped1 = cv2.resize(warped1[top:bottom], (W, H))
    warped2 = cv2.resize(warped2[top:bottom], (W, H))
    EAof2 = np.zeros((H, W, 3), np.uint8)
    EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
    EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
    
    #blend
    b_blend=time.time()
    W_lbl = 120
    blend_level = 4
    
    mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
    
    warped1[:, int(W_remap / 2):W - int(W_remap /
                2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
    warped2[EAof2 == 0] = warped1[EAof2 == 0]
    '''
    blended = multi_band_blending(
                    warped1, warped2, mask, blend_level)
    '''
    blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
    blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
    blended = np.ones((H,W,3))
    blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
    blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
    blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
    blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
    blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
    e_blend=time.time()
    #print("Time for blending is %.4f" %(e_blend-b_blend))
    video.write(blended.astype(np.uint8))
    #cv2.imwrite('video_blending.jpg', blended.astype(np.uint8))
    #FCN
    epochs=40
    index=np.where(status.ravel()==1)
    inliers=matches[index[0]]
    train,label = deduplication(inliers)
    zipped_train = list(zip(train, label))
    dataset = np.int32([e for e in zipped_train])
    np.random.shuffle(dataset)
    train = dataset[:, 0]
    label = dataset[:, 1]
    b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
    test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
    label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1)
    M = torch.tensor(M_current,requires_grad=True)
    #M_best = torch.tensor(M_best,requires_grad=True)
    model = nn.Linear(3,3,bias=False)
    model.weight =nn.Parameter(M)
    criterion = nn.MSELoss()
    test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
    label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
    
    test_output = model(test)
    test_output = test_output/test_output[:,2].reshape(-1,1)
    best_loss = criterion(test_output,label_test)/label_test.shape[0]
    
    print("The loss for frame1 is %.4f" %(best_loss))
    #print("The loss of M_best for frame1 is %.4f" %(loss_test_b))
    shape_list = []
    shape_list.append(inliers.shape[0])
    #M_best = M_best.to(device)
    while i<count:
        i+=1
        ret1,frame1=cap1.read()
        ret2,frame2=cap2.read()
        
        if ret1 and ret2:
            
            cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
            cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
            
            #shift
                
            shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
            shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
            shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
            shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
            if len(shape_list)%4==0:
                inliers = np.delete(inliers, range(0,shape_list[0]),axis = 0)
                del(shape_list[0])
             
            if i%60==0:
                cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
                cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
                b_matching=time.time()
                matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
                matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
                matchesR = matchesR[:, -1::-1]
                matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
                matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
                zipped_matches = list(zip(matchesL, matchesR))
                matches_append = np.int32([e for i in zipped_matches for e in i])
                pts1 = matches_append[:, 0]
                pts2 = matches_append[:, 1]
                
                
                e_matching=time.time()
                print("Time for matching is %.4f, round is %d" %((e_matching-b_matching),i) )
                M_, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
                index=np.where(status.ravel()==1)
      
                inliers = np.concatenate((inliers,matches_append[index[0]]),axis=0)
                shape_list.append(matches_append[index[0]].shape[0])
                
        
                train,label = deduplication(inliers)
                zipped_train = list(zip(train, label))
                dataset = np.int32([e for e in zipped_train])
                np.random.shuffle(dataset)
                train = dataset[:, 0]
                label = dataset[:, 1]
                
                b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
                test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                b=np.ones(int(train.shape[0]*0.8))
                train = np.insert(train[:int(train.shape[0]*0.8)], 2, values=b, axis=1) 
                label = np.insert(label[:int(label.shape[0]*0.8)], 2, values=b, axis=1) 
                
                M = torch.tensor(M_current,requires_grad=True)
                model = nn.Linear(3,3,bias=False)
                model.weight =nn.Parameter(M)
                model.to(device)
                criterion = nn.MSELoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
                #optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
                #M = torch.tensor(M,requires_grad=True)
                inputs = torch.tensor(train,requires_grad=False,dtype=torch.float64)
                inputs = inputs.to(device)
                label_t = torch.tensor(label,requires_grad=False,dtype=torch.float64)
                label_t = label_t.to(device)
                test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
                test = test.to(device)
                label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
                label_test = label_test.to(device)
                torch_dataset = Data.TensorDataset(inputs, label_t)

                loader = Data.DataLoader(
                                dataset=torch_dataset,      
                                batch_size=64,      
                                shuffle=True,               
                                num_workers=0,)
                total_step = len(loader)
                t_s = time.time()
                for epoch in range(epochs):
                    for step, (batch_x, l) in enumerate(loader):
                        #batch_x, l = batch_x.to(device),l.to(device)
                        outputs = model(batch_x)
                        outputs = outputs/outputs[:,2].reshape(-1,1)
                        loss = criterion(outputs,l)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, step+1, total_step, loss.item()/len(batch_x)))
                t_e = time.time()
                print(" Time for training is %.4f ms" %((t_e-t_s)*1000))
                test_output = model(test)
                test_output = test_output/test_output[:,2].reshape(-1,1)
                loss_test = criterion(test_output,label_test)/label_test.shape[0]
                
                M_c = torch.tensor(M_current,requires_grad=True)
                model_c = nn.Linear(3,3,bias=False)
                model_c.weight =nn.Parameter(M_c)
                model_c.to(device)
                current_test = model_c(test)
                current_test = current_test/current_test[:,2].reshape(-1,1)
                current_loss = criterion(current_test,label_test)/label_test.shape[0]
                
                print("The loss of M_trained is %.4f, round %d" %(loss_test,i))
                print("The loss of M_current is %.4f, round %d" %(current_loss,i))
                print(model.weight)
                
                if loss_test<current_loss:
                    model = model.cpu()
                    M_current = model.weight.detach().numpy()
                    M_current = M_current/M_current[2][2]
                    best_loss = loss_test
                    print("The best loss is %.4f" %(best_loss))
            
            top, bottom = verticalBoundary(M_current, W_remap, W, H)
            warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
            warped1 = shifted_cams[0:H,:]
            warped1 = cv2.resize(warped1[top:bottom], (W, H))
            warped2 = cv2.resize(warped2[top:bottom], (W, H))
            EAof2 = np.zeros((H, W, 3), np.uint8)
            EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
            EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
            b_blend=time.time()
            
            if i%240==0:
                mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
            
            
            
            warped1[:, int(W_remap / 2):W - int(W_remap /
                        2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
            warped2[EAof2 == 0] = warped1[EAof2 == 0]
            '''
            blended = multi_band_blending(
                                warped1, warped2, mask, blend_level)
            '''
            #blended = warped1 * mask + warped2 * (1 - mask)
            blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
            blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
            blended = np.ones((H,W,3))
            blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
            blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
            blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
            blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
            blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
            video.write(blended.astype(np.uint8))
            e_blend=time.time()
            print(" Time for blending is %.4f, round is %d" %((e_blend-b_blend),i) )
      
        else:   
            break
    
else:
    print('fail')
cap1.release()
cap2.release()
video.release()

successful
The count is 943
The loss for frame1 is 0.0421
 Time for blending is 0.3495, round is 2
 Time for blending is 0.3383, round is 3
 Time for blending is 0.3079, round is 4
 Time for blending is 0.3351, round is 5
 Time for blending is 0.3189, round is 6
 Time for blending is 0.3244, round is 7
 Time for blending is 0.3228, round is 8
 Time for blending is 0.3151, round is 9
 Time for blending is 0.3117, round is 10
 Time for blending is 0.3151, round is 11
 Time for blending is 0.3153, round is 12
 Time for blending is 0.3135, round is 13
 Time for blending is 0.3096, round is 14
 Time for blending is 0.3158, round is 15
 Time for blending is 0.3204, round is 16
 Time for blending is 0.3310, round is 17
 Time for blending is 0.3208, round is 18
 Time for blending is 0.3269, round is 19
 Time for blending is 0.3279, round is 20
 Time for blending is 0.3154, round is 21
 Time for blending is 0.3285, round is 22
 Time for blending is 0.3319, round is 23
 Time for blending is 0.32

 Time for blending is 0.3077, round is 60
 Time for blending is 0.3053, round is 61
 Time for blending is 0.3059, round is 62
 Time for blending is 0.3125, round is 63
 Time for blending is 0.3149, round is 64
 Time for blending is 0.3128, round is 65
 Time for blending is 0.3001, round is 66
 Time for blending is 0.3136, round is 67
 Time for blending is 0.3119, round is 68
 Time for blending is 0.3058, round is 69
 Time for blending is 0.3069, round is 70
 Time for blending is 0.3332, round is 71
 Time for blending is 0.2889, round is 72
 Time for blending is 0.3256, round is 73
 Time for blending is 0.3154, round is 74
 Time for blending is 0.2981, round is 75
 Time for blending is 0.3134, round is 76
 Time for blending is 0.2984, round is 77
 Time for blending is 0.3250, round is 78
 Time for blending is 0.3253, round is 79
 Time for blending is 0.3223, round is 80
 Time for blending is 0.3062, round is 81
 Time for blending is 0.3631, round is 82
 Time for blending is 0.3189, roun

Epoch [32/40], Step [1/6], Loss: 0.0889
Epoch [32/40], Step [2/6], Loss: 0.0466
Epoch [32/40], Step [3/6], Loss: 0.0640
Epoch [32/40], Step [4/6], Loss: 0.0863
Epoch [32/40], Step [5/6], Loss: 0.0525
Epoch [32/40], Step [6/6], Loss: 0.2078
Epoch [33/40], Step [1/6], Loss: 0.0542
Epoch [33/40], Step [2/6], Loss: 0.0730
Epoch [33/40], Step [3/6], Loss: 0.0498
Epoch [33/40], Step [4/6], Loss: 0.0515
Epoch [33/40], Step [5/6], Loss: 0.0457
Epoch [33/40], Step [6/6], Loss: 0.2284
Epoch [34/40], Step [1/6], Loss: 0.0384
Epoch [34/40], Step [2/6], Loss: 0.0526
Epoch [34/40], Step [3/6], Loss: 0.0543
Epoch [34/40], Step [4/6], Loss: 0.0727
Epoch [34/40], Step [5/6], Loss: 0.0622
Epoch [34/40], Step [6/6], Loss: 0.1994
Epoch [35/40], Step [1/6], Loss: 0.0727
Epoch [35/40], Step [2/6], Loss: 0.0576
Epoch [35/40], Step [3/6], Loss: 0.0732
Epoch [35/40], Step [4/6], Loss: 0.0543
Epoch [35/40], Step [5/6], Loss: 0.0523
Epoch [35/40], Step [6/6], Loss: 0.1181
Epoch [36/40], Step [1/6], Loss: 0.0492


Epoch [26/40], Step [1/7], Loss: 0.0574
Epoch [26/40], Step [2/7], Loss: 0.0443
Epoch [26/40], Step [3/7], Loss: 0.0624
Epoch [26/40], Step [4/7], Loss: 0.0718
Epoch [26/40], Step [5/7], Loss: 0.0482
Epoch [26/40], Step [6/7], Loss: 0.0659
Epoch [26/40], Step [7/7], Loss: 0.1208
Epoch [27/40], Step [1/7], Loss: 0.0448
Epoch [27/40], Step [2/7], Loss: 0.0652
Epoch [27/40], Step [3/7], Loss: 0.0473
Epoch [27/40], Step [4/7], Loss: 0.0566
Epoch [27/40], Step [5/7], Loss: 0.0719
Epoch [27/40], Step [6/7], Loss: 0.0791
Epoch [27/40], Step [7/7], Loss: 0.1105
Epoch [28/40], Step [1/7], Loss: 0.0928
Epoch [28/40], Step [2/7], Loss: 0.0498
Epoch [28/40], Step [3/7], Loss: 0.0720
Epoch [28/40], Step [4/7], Loss: 0.0568
Epoch [28/40], Step [5/7], Loss: 0.0816
Epoch [28/40], Step [6/7], Loss: 0.0511
Epoch [28/40], Step [7/7], Loss: 0.1446
Epoch [29/40], Step [1/7], Loss: 0.0519
Epoch [29/40], Step [2/7], Loss: 0.0612
Epoch [29/40], Step [3/7], Loss: 0.0523
Epoch [29/40], Step [4/7], Loss: 0.0766


Epoch [20/40], Step [4/7], Loss: 0.0589
Epoch [20/40], Step [5/7], Loss: 0.0797
Epoch [20/40], Step [6/7], Loss: 0.0494
Epoch [20/40], Step [7/7], Loss: 0.1577
Epoch [21/40], Step [1/7], Loss: 0.0510
Epoch [21/40], Step [2/7], Loss: 0.1084
Epoch [21/40], Step [3/7], Loss: 0.0784
Epoch [21/40], Step [4/7], Loss: 0.0751
Epoch [21/40], Step [5/7], Loss: 0.1063
Epoch [21/40], Step [6/7], Loss: 0.0881
Epoch [21/40], Step [7/7], Loss: 0.1549
Epoch [22/40], Step [1/7], Loss: 0.0813
Epoch [22/40], Step [2/7], Loss: 0.0736
Epoch [22/40], Step [3/7], Loss: 0.0723
Epoch [22/40], Step [4/7], Loss: 0.0590
Epoch [22/40], Step [5/7], Loss: 0.0793
Epoch [22/40], Step [6/7], Loss: 0.0736
Epoch [22/40], Step [7/7], Loss: 0.1507
Epoch [23/40], Step [1/7], Loss: 0.0622
Epoch [23/40], Step [2/7], Loss: 0.0744
Epoch [23/40], Step [3/7], Loss: 0.0952
Epoch [23/40], Step [4/7], Loss: 0.0486
Epoch [23/40], Step [5/7], Loss: 0.0790
Epoch [23/40], Step [6/7], Loss: 0.0678
Epoch [23/40], Step [7/7], Loss: 0.2061


...

 Time for blending is 0.3113, round is 660
 Time for blending is 0.3079, round is 661
 Time for blending is 0.3104, round is 662
 Time for blending is 0.3116, round is 663
 Time for blending is 0.3424, round is 664
 Time for blending is 0.3131, round is 665
 Time for blending is 0.3338, round is 666
 Time for blending is 0.3140, round is 667
 Time for blending is 0.3258, round is 668
 Time for blending is 0.3127, round is 669
 Time for blending is 0.3269, round is 670
 Time for blending is 0.3275, round is 671
 Time for blending is 0.3274, round is 672
 Time for blending is 0.3475, round is 673
 Time for blending is 0.3160, round is 674
 Time for blending is 0.3252, round is 675
 Time for blending is 0.3278, round is 676
 Time for blending is 0.3266, round is 677
 Time for blending is 0.3340, round is 678
 Time for blending is 0.3223, round is 679
 Time for blending is 0.3370, round is 680
 Time for blending is 0.3344, round is 681
 Time for blending is 0.3196, round is 682
 Time for b

 Time for blending is 0.8009, round is 720
 Time for blending is 0.3079, round is 721
 Time for blending is 0.3095, round is 722
 Time for blending is 0.3111, round is 723
 Time for blending is 0.3223, round is 724
 Time for blending is 0.3055, round is 725
 Time for blending is 0.3132, round is 726
 Time for blending is 0.3001, round is 727
 Time for blending is 0.3096, round is 728
 Time for blending is 0.3271, round is 729
 Time for blending is 0.3179, round is 730
 Time for blending is 0.3216, round is 731
 Time for blending is 0.3176, round is 732
 Time for blending is 0.3198, round is 733
 Time for blending is 0.2997, round is 734
 Time for blending is 0.3214, round is 735
 Time for blending is 0.3302, round is 736
 Time for blending is 0.3106, round is 737
 Time for blending is 0.3279, round is 738
 Time for blending is 0.3115, round is 739
 Time for blending is 0.3500, round is 740
 Time for blending is 0.3178, round is 741
 Time for blending is 0.3122, round is 742
 Time for b

Epoch [38/40], Step [5/6], Loss: 0.1686
Epoch [38/40], Step [6/6], Loss: 0.1755
Epoch [39/40], Step [1/6], Loss: 0.1583
Epoch [39/40], Step [2/6], Loss: 0.1685
Epoch [39/40], Step [3/6], Loss: 0.1934
Epoch [39/40], Step [4/6], Loss: 0.2026
Epoch [39/40], Step [5/6], Loss: 0.2178
Epoch [39/40], Step [6/6], Loss: 0.2789
Epoch [40/40], Step [1/6], Loss: 0.1590
Epoch [40/40], Step [2/6], Loss: 0.1587
Epoch [40/40], Step [3/6], Loss: 0.2780
Epoch [40/40], Step [4/6], Loss: 0.1443
Epoch [40/40], Step [5/6], Loss: 0.2279
Epoch [40/40], Step [6/6], Loss: 0.2227
 Time for training is 613.3935 ms
The loss of M_trained is 0.1333, round 780
The loss of M_current is 0.1133, round 780
Parameter containing:
tensor([[ 9.9004e-01, -1.2393e-02,  3.5323e+01],
        [-8.7521e-03,  9.9549e-01,  2.4158e+01],
        [-1.9055e-06,  7.1245e-08,  1.0000e+00]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)
 Time for blending is 0.3109, round is 780
 Time for blending is 0.3060, round is 781

Epoch [27/40], Step [3/6], Loss: 0.1187
Epoch [27/40], Step [4/6], Loss: 0.2496
Epoch [27/40], Step [5/6], Loss: 0.1734
Epoch [27/40], Step [6/6], Loss: 0.1606
Epoch [28/40], Step [1/6], Loss: 0.1607
Epoch [28/40], Step [2/6], Loss: 0.3027
Epoch [28/40], Step [3/6], Loss: 0.1049
Epoch [28/40], Step [4/6], Loss: 0.2004
Epoch [28/40], Step [5/6], Loss: 0.2498
Epoch [28/40], Step [6/6], Loss: 0.2721
Epoch [29/40], Step [1/6], Loss: 0.1750
Epoch [29/40], Step [2/6], Loss: 0.2257
Epoch [29/40], Step [3/6], Loss: 0.1237
Epoch [29/40], Step [4/6], Loss: 0.1598
Epoch [29/40], Step [5/6], Loss: 0.1260
Epoch [29/40], Step [6/6], Loss: 0.2138
Epoch [30/40], Step [1/6], Loss: 0.1628
Epoch [30/40], Step [2/6], Loss: 0.1471
Epoch [30/40], Step [3/6], Loss: 0.1553
Epoch [30/40], Step [4/6], Loss: 0.1186
Epoch [30/40], Step [5/6], Loss: 0.1741
Epoch [30/40], Step [6/6], Loss: 0.3022
Epoch [31/40], Step [1/6], Loss: 0.1758
Epoch [31/40], Step [2/6], Loss: 0.1471
Epoch [31/40], Step [3/6], Loss: 0.1514


Epoch [30/40], Step [3/6], Loss: 0.3269
Epoch [30/40], Step [4/6], Loss: 0.2234
Epoch [30/40], Step [5/6], Loss: 0.1450
Epoch [30/40], Step [6/6], Loss: 0.2555
Epoch [31/40], Step [1/6], Loss: 0.2030
Epoch [31/40], Step [2/6], Loss: 0.2183
Epoch [31/40], Step [3/6], Loss: 0.1287
Epoch [31/40], Step [4/6], Loss: 0.2618
Epoch [31/40], Step [5/6], Loss: 0.1852
Epoch [31/40], Step [6/6], Loss: 0.1994
Epoch [32/40], Step [1/6], Loss: 0.1862
Epoch [32/40], Step [2/6], Loss: 0.2158
Epoch [32/40], Step [3/6], Loss: 0.1850
Epoch [32/40], Step [4/6], Loss: 0.1643
Epoch [32/40], Step [5/6], Loss: 0.2549
Epoch [32/40], Step [6/6], Loss: 0.1625
Epoch [33/40], Step [1/6], Loss: 0.1316
Epoch [33/40], Step [2/6], Loss: 0.2053
Epoch [33/40], Step [3/6], Loss: 0.1260
Epoch [33/40], Step [4/6], Loss: 0.2151
Epoch [33/40], Step [5/6], Loss: 0.2465
Epoch [33/40], Step [6/6], Loss: 0.2539
Epoch [34/40], Step [1/6], Loss: 0.2117
Epoch [34/40], Step [2/6], Loss: 0.1592
Epoch [34/40], Step [3/6], Loss: 0.2061


In [13]:
import torch
import torch.optim as optim
from torch import nn
import torch.utils.data as Data

cap1=cv2.VideoCapture('C:/Users/Jackie/Downloads/0119_1.MP4')
cap2=cv2.VideoCapture('C:/Users/Jackie/Downloads/0119_2.MP4')
count=cap1.get(cv2.CAP_PROP_FRAME_COUNT)
fps=cap1.get(cv2.CAP_PROP_FPS)
size=(int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH)*2),int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video=cv2.VideoWriter('C:/Users/Jackie/Downloads/119_200fov.MP4',cv2.VideoWriter_fourcc('I','4','2','0'),fps,size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



maxL = 200
maxR = 200

if cap1.isOpened() and cap2.isOpened():
    i=1
    print('successful')
    print("The count is %d" %(count))
    #image size paremeters
    
        
    ret1,frame1=cap1.read()
    ret2,frame2=cap2.read()
    H=frame1.shape[0]
    W_remap=int(200/180*H)
    W=frame1.shape[1]+frame2.shape[1]
    templ_shape = (60, 16)
    offsetYL = int(160/1280*H)
    offsetYR = int(160/1280*H)
    #print(W_remap)
    #remap
    xmap, ymap, p = buildmap(Ws=W_remap, Hs=H, Wd=H, Hd=H, fov=200)
    #cam2 = cv2.remap(frame[:, :H], xmap, ymap, cv2.INTER_LINEAR)
    #cam1 = cv2.remap(frame[:, H:], xmap, ymap, cv2.INTER_LINEAR)
    cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
    cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
    
    #shift
    cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
    cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
    shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
    shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
    shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
    shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
    
    #cv2.imwrite('video_shift.jpg', shifted_cams)
    #matching
    
    maxL = 200
    maxR = 200
    b_matching=time.time()
    matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
    matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
    matchesR = matchesR[:, -1::-1]
    matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
    matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
    zipped_matches = list(zip(matchesL, matchesR))
    matches = np.int32([e for i in zipped_matches for e in i])
    pts1 = matches[:, 0]
    pts2 = matches[:, 1]
    
    #homography
    M_current, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
    
    top, bottom = verticalBoundary(M_current, W_remap, W, H)
    warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
    warped1 = shifted_cams[0:H,:]
    warped1 = cv2.resize(warped1[top:bottom], (W, H))
    warped2 = cv2.resize(warped2[top:bottom], (W, H))
    EAof2 = np.zeros((H, W, 3), np.uint8)
    EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
    EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
    
    #blend
    b_blend=time.time()
    W_lbl = 192
    blend_level = 5
    
    mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
    
    warped1[:, int(W_remap / 2):W - int(W_remap /
                2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
    warped2[EAof2 == 0] = warped1[EAof2 == 0]
    '''
    blended = multi_band_blending(
                    warped1, warped2, mask, blend_level)
    '''
    blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
    blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
    blended = np.ones((H,W,3))
    blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
    blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
    blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
    blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
    blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
    e_blend=time.time()
    #print("Time for blending is %.4f" %(e_blend-b_blend))
    video.write(blended.astype(np.uint8))
    #cv2.imwrite('video_blending.jpg', blended.astype(np.uint8))
    #FCN
    epochs=40
    index=np.where(status.ravel()==1)
    inliers=matches[index[0]]
    train,label = deduplication(inliers)
    zipped_train = list(zip(train, label))
    dataset = np.int32([e for e in zipped_train])
    np.random.shuffle(dataset)
    train = dataset[:, 0]
    label = dataset[:, 1]
    b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
    test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
    label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1)
    M = torch.tensor(M_current,requires_grad=True)
    #M_best = torch.tensor(M_best,requires_grad=True)
    model = nn.Linear(3,3,bias=False)
    model.weight =nn.Parameter(M)
    criterion = nn.MSELoss()
    test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
    label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
    
    test_output = model(test)
    test_output = test_output/test_output[:,2].reshape(-1,1)
    best_loss = criterion(test_output,label_test)/label_test.shape[0]
    
    print("The loss for frame1 is %.4f" %(best_loss))
    #print("The loss of M_best for frame1 is %.4f" %(loss_test_b))
    shape_list = []
    shape_list.append(inliers.shape[0])
    #M_best = M_best.to(device)
    while i<count:
        i+=1
        ret1,frame1=cap1.read()
        ret2,frame2=cap2.read()
        
        if ret1 and ret2:
            
            cam1 = cv2.remap(frame1, xmap, ymap, cv2.INTER_LINEAR)
            cam2 = cv2.remap(frame2, xmap, ymap, cv2.INTER_LINEAR)
            
            #shift
                
            shifted_cams = np.zeros((H * 2, W, 3), np.uint8)
            shifted_cams[H:, int((W - W_remap) / 2):int((W + W_remap) / 2)] = cam1
            shifted_cams[:H, :int(W_remap / 2)] = cam2[:, int(W_remap / 2):]
            shifted_cams[:H, W - int(W_remap / 2):] = cam2[:, :int(W_remap / 2)]
            if len(shape_list)%4==0:
                inliers = np.delete(inliers, range(0,shape_list[0]),axis = 0)
                del(shape_list[0])
             
            if i%60==0:
                cam1_gray = cv2.cvtColor(cam1, cv2.COLOR_BGR2GRAY)
                cam2_gray = cv2.cvtColor(cam2, cv2.COLOR_BGR2GRAY)
                b_matching=time.time()
                matchesL = getMatches_goodtemplmatch(
                    cam1_gray[offsetYL:H - offsetYL, int(W / 2):],
                    cam2_gray[offsetYL:H - offsetYL, :W_remap - int(W / 2)],
                    templ_shape, maxL)
                matchesR = getMatches_goodtemplmatch(
                    cam2_gray[offsetYR:H - offsetYR, int(W / 2):],
                    cam1_gray[offsetYR:H - offsetYR, :W_remap - int(W / 2)],
                    templ_shape, maxR)
                matchesR = matchesR[:, -1::-1]
                matchesL = matchesL + (int((W - W_remap) / 2), offsetYL)
                matchesR = matchesR + (int((W - W_remap) / 2) + int(W / 2), offsetYR)
                zipped_matches = list(zip(matchesL, matchesR))
                matches_append = np.int32([e for i in zipped_matches for e in i])
                pts1 = matches_append[:, 0]
                pts2 = matches_append[:, 1]
                
                
                e_matching=time.time()
                print("Time for matching is %.4f, round is %d" %((e_matching-b_matching),i) )
                M_, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)
                index=np.where(status.ravel()==1)
      
                inliers = np.concatenate((inliers,matches_append[index[0]]),axis=0)
                shape_list.append(matches_append[index[0]].shape[0])
                
        
                train,label = deduplication(inliers)
                zipped_train = list(zip(train, label))
                dataset = np.int32([e for e in zipped_train])
                np.random.shuffle(dataset)
                train = dataset[:, 0]
                label = dataset[:, 1]
                
                b_test = np.ones(int(train[int(train.shape[0]*0.8):].shape[0]))
                test = np.insert(train[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                label_test = np.insert(label[int(train.shape[0]*0.8):], 2, values=b_test, axis=1) 
                b=np.ones(int(train.shape[0]*0.8))
                train = np.insert(train[:int(train.shape[0]*0.8)], 2, values=b, axis=1) 
                label = np.insert(label[:int(label.shape[0]*0.8)], 2, values=b, axis=1) 
                
                M = torch.tensor(M_current,requires_grad=True)
                model = nn.Linear(3,3,bias=False)
                model.weight =nn.Parameter(M)
                model.to(device)
                criterion = nn.MSELoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
                #optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
                #M = torch.tensor(M,requires_grad=True)
                inputs = torch.tensor(train,requires_grad=False,dtype=torch.float64)
                inputs = inputs.to(device)
                label_t = torch.tensor(label,requires_grad=False,dtype=torch.float64)
                label_t = label_t.to(device)
                test = torch.tensor(test,requires_grad=False,dtype=torch.float64)
                test = test.to(device)
                label_test = torch.tensor(label_test,requires_grad=False,dtype=torch.float64)
                label_test = label_test.to(device)
                torch_dataset = Data.TensorDataset(inputs, label_t)

                loader = Data.DataLoader(
                                dataset=torch_dataset,      
                                batch_size=64,      
                                shuffle=True,               
                                num_workers=0,)
                total_step = len(loader)
                t_s = time.time()
                for epoch in range(epochs):
                    for step, (batch_x, l) in enumerate(loader):
                        #batch_x, l = batch_x.to(device),l.to(device)
                        outputs = model(batch_x)
                        outputs = outputs/outputs[:,2].reshape(-1,1)
                        loss = criterion(outputs,l)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, step+1, total_step, loss.item()/len(batch_x)))
                t_e = time.time()
                print(" Time for training is %.4f ms" %((t_e-t_s)*1000))
                test_output = model(test)
                test_output = test_output/test_output[:,2].reshape(-1,1)
                loss_test = criterion(test_output,label_test)/label_test.shape[0]
                
                M_c = torch.tensor(M_current,requires_grad=True)
                model_c = nn.Linear(3,3,bias=False)
                model_c.weight =nn.Parameter(M_c)
                model_c.to(device)
                current_test = model_c(test)
                current_test = current_test/current_test[:,2].reshape(-1,1)
                current_loss = criterion(current_test,label_test)/label_test.shape[0]
                
                print("The loss of M_trained is %.4f, round %d" %(loss_test,i))
                print("The loss of M_current is %.4f, round %d" %(current_loss,i))
                print(model.weight)
                
                if loss_test<current_loss:
                    model = model.cpu()
                    M_current = model.weight.detach().numpy()
                    M_current = M_current/M_current[2][2]
                    best_loss = loss_test
                    print("The best loss is %.4f" %(best_loss))
            
            top, bottom = verticalBoundary(M_current, W_remap, W, H)
            warped2 = cv2.warpPerspective(shifted_cams[H:], M_current, (W, H))
            warped1 = shifted_cams[0:H,:]
            warped1 = cv2.resize(warped1[top:bottom], (W, H))
            warped2 = cv2.resize(warped2[top:bottom], (W, H))
            EAof2 = np.zeros((H, W, 3), np.uint8)
            EAof2[:, int((W - W_remap) / 2) + 1:int((W + W_remap) / 2) - 1] = 255
            EAof2 = cv2.warpPerspective(EAof2, M_current, (W, H))
            #b_blend=time.time()
            e1 = cv2.getTickCount()
            if i%120==0:
                mask, minloc_old = imgLabeling(
                warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)],
                warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl],
                (W, H), int(W_remap / 2) - W_lbl, W - int(W_remap / 2))
            
            
            
            warped1[:, int(W_remap / 2):W - int(W_remap /
                        2)] = warped2[:, int(W_remap / 2):W - int(W_remap / 2)]
            warped2[EAof2 == 0] = warped1[EAof2 == 0]
            '''
            blended = multi_band_blending(
                                warped1, warped2, mask, blend_level)
            '''
            #blended = warped1 * mask + warped2 * (1 - mask)
           
            blended_l = multi_band_blending(
                    warped1[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    warped2[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    mask[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)], 
                    blend_level)
                
            blended_r = multi_band_blending(
                    warped1[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    warped2[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    mask[:, W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl], 
                    blend_level)
            blended = np.ones((H,W,3))
            blended[:, 0:int(W_remap / 2) - W_lbl] = warped1[:, 0:int(W_remap / 2) - W_lbl]
            blended[:, int(W_remap / 2) - W_lbl:int(W_remap / 2)] = blended_l
            blended[:,int(W_remap / 2):W - int(W_remap / 2)] = warped2[:,int(W_remap / 2):W - int(W_remap / 2)]
            blended[:,W - int(W_remap / 2):W - int(W_remap / 2) + W_lbl] = blended_r
            blended[:,W - int(W_remap / 2) + W_lbl:] = warped1[:,W - int(W_remap / 2) + W_lbl:]
            video.write(blended.astype(np.uint8))
            e2 = cv2.getTickCount()
            #e_blend=time.time()
            #print(" Time for blending is %.4f, round is %d" %((e_blend-b_blend),i) )
            print("Time for blending is %.4f, round is %d" %(((e2 - e1)/cv2.getTickFrequency()*1000),i))
      
        else:   
            break
    
else:
    print('fail')
cap1.release()
cap2.release()
video.release()

successful
The count is 636
The loss for frame1 is 0.1454
Time for blending is 351.6364, round is 2
Time for blending is 355.8939, round is 3
Time for blending is 351.8112, round is 4
Time for blending is 354.7360, round is 5
Time for blending is 362.4592, round is 6
Time for blending is 359.3792, round is 7
Time for blending is 358.0967, round is 8
Time for blending is 361.4350, round is 9
Time for blending is 354.1075, round is 10
Time for blending is 363.6786, round is 11
Time for blending is 358.0551, round is 12
Time for blending is 355.6982, round is 13
Time for blending is 363.6397, round is 14
Time for blending is 370.4077, round is 15
Time for blending is 394.6088, round is 16
Time for blending is 436.8062, round is 17
Time for blending is 353.8472, round is 18
Time for blending is 362.7728, round is 19
Time for blending is 366.5807, round is 20
Time for blending is 372.0474, round is 21
Time for blending is 361.2201, round is 22
Time for blending is 375.5492, round is 23
Time

Time for blending is 350.7620, round is 108
Time for blending is 349.0496, round is 109
Time for blending is 351.1217, round is 110
Time for blending is 362.5821, round is 111
Time for blending is 362.6510, round is 112
Time for blending is 368.4492, round is 113
Time for blending is 363.0359, round is 114
Time for blending is 360.4545, round is 115
Time for blending is 360.3970, round is 116
Time for blending is 358.2074, round is 117
Time for blending is 356.9756, round is 118
Time for blending is 359.2382, round is 119
Time for matching is 3.5499, round is 120
Epoch [1/40], Step [1/2], Loss: 0.0778
Epoch [1/40], Step [2/2], Loss: 2.4709
Epoch [2/40], Step [1/2], Loss: 0.1309
Epoch [2/40], Step [2/2], Loss: 0.5831
Epoch [3/40], Step [1/2], Loss: 0.5635
Epoch [3/40], Step [2/2], Loss: 1.1590
Epoch [4/40], Step [1/2], Loss: 0.1540
Epoch [4/40], Step [2/2], Loss: 0.0896
Epoch [5/40], Step [1/2], Loss: 0.2191
Epoch [5/40], Step [2/2], Loss: 0.9656
Epoch [6/40], Step [1/2], Loss: 0.2840
E

Epoch [23/40], Step [2/2], Loss: 0.0637
Epoch [24/40], Step [1/2], Loss: 0.0485
Epoch [24/40], Step [2/2], Loss: 0.1379
Epoch [25/40], Step [1/2], Loss: 0.0504
Epoch [25/40], Step [2/2], Loss: 0.1191
Epoch [26/40], Step [1/2], Loss: 0.0506
Epoch [26/40], Step [2/2], Loss: 0.1289
Epoch [27/40], Step [1/2], Loss: 0.0889
Epoch [27/40], Step [2/2], Loss: 0.0619
Epoch [28/40], Step [1/2], Loss: 0.0463
Epoch [28/40], Step [2/2], Loss: 0.1283
Epoch [29/40], Step [1/2], Loss: 0.0611
Epoch [29/40], Step [2/2], Loss: 0.1062
Epoch [30/40], Step [1/2], Loss: 0.0525
Epoch [30/40], Step [2/2], Loss: 0.1151
Epoch [31/40], Step [1/2], Loss: 0.0720
Epoch [31/40], Step [2/2], Loss: 0.0877
Epoch [32/40], Step [1/2], Loss: 0.0605
Epoch [32/40], Step [2/2], Loss: 0.1020
Epoch [33/40], Step [1/2], Loss: 0.0812
Epoch [33/40], Step [2/2], Loss: 0.0732
Epoch [34/40], Step [1/2], Loss: 0.0806
Epoch [34/40], Step [2/2], Loss: 0.0728
Epoch [35/40], Step [1/2], Loss: 0.0510
Epoch [35/40], Step [2/2], Loss: 0.1164


Time for blending is 358.1407, round is 244
Time for blending is 369.5022, round is 245
Time for blending is 368.9925, round is 246
Time for blending is 367.9943, round is 247
Time for blending is 370.1484, round is 248
Time for blending is 363.0771, round is 249
Time for blending is 369.3234, round is 250
Time for blending is 367.2690, round is 251
Time for blending is 356.4604, round is 252
Time for blending is 369.1075, round is 253
Time for blending is 363.6017, round is 254
Time for blending is 349.5879, round is 255
Time for blending is 351.6605, round is 256
Time for blending is 353.8710, round is 257
Time for blending is 347.5373, round is 258
Time for blending is 355.3331, round is 259
Time for blending is 355.4628, round is 260
Time for blending is 360.4406, round is 261
Time for blending is 354.2036, round is 262
Time for blending is 366.4657, round is 263
Time for blending is 357.0239, round is 264
Time for blending is 353.5811, round is 265
Time for blending is 358.1812, r

Time for blending is 359.8515, round is 349
Time for blending is 355.5352, round is 350
Time for blending is 368.6318, round is 351
Time for blending is 353.6619, round is 352
Time for blending is 360.7755, round is 353
Time for blending is 356.7302, round is 354
Time for blending is 363.5914, round is 355
Time for blending is 369.8379, round is 356
Time for blending is 371.0443, round is 357
Time for blending is 362.4005, round is 358
Time for blending is 377.1155, round is 359
Time for matching is 3.6478, round is 360
Epoch [1/40], Step [1/2], Loss: 0.2840
Epoch [1/40], Step [2/2], Loss: 1.3037
Epoch [2/40], Step [1/2], Loss: 0.3156
Epoch [2/40], Step [2/2], Loss: 0.5564
Epoch [3/40], Step [1/2], Loss: 0.6153
Epoch [3/40], Step [2/2], Loss: 0.7725
Epoch [4/40], Step [1/2], Loss: 0.2634
Epoch [4/40], Step [2/2], Loss: 0.3302
Epoch [5/40], Step [1/2], Loss: 0.3469
Epoch [5/40], Step [2/2], Loss: 0.6551
Epoch [6/40], Step [1/2], Loss: 0.5016
Epoch [6/40], Step [2/2], Loss: 0.2006
Epoch 

Time for blending is 347.2837, round is 420
Time for blending is 349.1919, round is 421
Time for blending is 353.0423, round is 422
Time for blending is 365.4904, round is 423
Time for blending is 350.6913, round is 424
Time for blending is 384.7481, round is 425
Time for blending is 351.9161, round is 426
Time for blending is 362.8605, round is 427
Time for blending is 357.3090, round is 428
Time for blending is 354.9068, round is 429
Time for blending is 352.5229, round is 430
Time for blending is 356.0060, round is 431
Time for blending is 348.9612, round is 432
Time for blending is 353.6928, round is 433
Time for blending is 347.8603, round is 434
Time for blending is 352.0141, round is 435
Time for blending is 365.6496, round is 436
Time for blending is 362.4947, round is 437
Time for blending is 360.9427, round is 438
Time for blending is 358.7888, round is 439
Time for blending is 361.7369, round is 440
Time for blending is 355.4216, round is 441
Time for blending is 352.2557, r

Time for blending is 356.0227, round is 525
Time for blending is 375.7836, round is 526
Time for blending is 352.1705, round is 527
Time for blending is 356.4446, round is 528
Time for blending is 375.6604, round is 529
Time for blending is 350.5439, round is 530
Time for blending is 377.9872, round is 531
Time for blending is 350.9644, round is 532
Time for blending is 348.8188, round is 533
Time for blending is 350.6868, round is 534
Time for blending is 356.9715, round is 535
Time for blending is 352.3243, round is 536
Time for blending is 353.0014, round is 537
Time for blending is 348.9266, round is 538
Time for blending is 353.2227, round is 539
Time for matching is 3.6753, round is 540
Epoch [1/40], Step [1/2], Loss: 1.0124
Epoch [1/40], Step [2/2], Loss: 1.5429
Epoch [2/40], Step [1/2], Loss: 1.0295
Epoch [2/40], Step [2/2], Loss: 0.5397
Epoch [3/40], Step [1/2], Loss: 1.4752
Epoch [3/40], Step [2/2], Loss: 0.6374
Epoch [4/40], Step [1/2], Loss: 0.6098
Epoch [4/40], Step [2/2],

Epoch [22/40], Step [2/2], Loss: 1.1949
Epoch [23/40], Step [1/2], Loss: 0.9610
Epoch [23/40], Step [2/2], Loss: 0.3132
Epoch [24/40], Step [1/2], Loss: 0.9312
Epoch [24/40], Step [2/2], Loss: 0.3246
Epoch [25/40], Step [1/2], Loss: 0.9569
Epoch [25/40], Step [2/2], Loss: 0.3213
Epoch [26/40], Step [1/2], Loss: 0.9505
Epoch [26/40], Step [2/2], Loss: 0.3348
Epoch [27/40], Step [1/2], Loss: 0.6205
Epoch [27/40], Step [2/2], Loss: 1.0244
Epoch [28/40], Step [1/2], Loss: 0.5860
Epoch [28/40], Step [2/2], Loss: 1.0907
Epoch [29/40], Step [1/2], Loss: 0.6186
Epoch [29/40], Step [2/2], Loss: 1.0009
Epoch [30/40], Step [1/2], Loss: 0.9045
Epoch [30/40], Step [2/2], Loss: 0.4073
Epoch [31/40], Step [1/2], Loss: 0.6190
Epoch [31/40], Step [2/2], Loss: 1.0088
Epoch [32/40], Step [1/2], Loss: 0.6124
Epoch [32/40], Step [2/2], Loss: 1.0189
Epoch [33/40], Step [1/2], Loss: 0.5653
Epoch [33/40], Step [2/2], Loss: 1.1118
Epoch [34/40], Step [1/2], Loss: 0.5483
Epoch [34/40], Step [2/2], Loss: 1.1569


In [9]:
cv2.useOptimized()

True

In [5]:
e1 = cv2.getTickCount()
for i in range(10):
    i=i+1
e2 = cv2.getTickCount()
print((e2 - e1)/cv2.getTickFrequency()*1000)

0.0361
